In [33]:
# load libraries
import pandas as pd
import os, re
import datetime as dt
from sqlalchemy import create_engine, text
import zipfile
# from dask import dataframe as dd

## Setup

In [55]:
# Filtering only for a certain type of route?
filter_by_route = True

In [56]:
# Welches Jahr?
jahr = "2021"              # year of interest
# Welcher Zip?
source = "gtfs.de" # where is the feed file? delfi or gtfs.de?
zipname = '2021_reissue_2' # name of GTFS zipfile

# define paths
workingdir = "../../data/" 
#storagedir = "smb://192.168.90.30/allmende%20verkehr/4%20Projekte/2%20Projekte%20Mobilitätswende/ÖV-Deutschlandkarte%20(Gap-Map)/Berechnungen/raw/gtfs/"

# constructed paths
rawdir = workingdir + "raw/" # where is all the data?
gtfsdir = rawdir + source + "/" # where zip-file is located
outdir = workingdir + "interim/" # where do outputfiles go?
zippath = gtfsdir + zipname + ".zip"

# set up zip file as default for functions
zf = zipfile.ZipFile(zippath) # this is the raw stuff

In [57]:
# choose file-based output connection
outpath = '{0}{1}.db'.format(outdir,zipname)
# set up DB connection
dbout = create_engine('sqlite:///' + outpath)

# Count service_ids

In [58]:
def interveningWeekdays(start, end, inclusive=True, weekdays=[0, 1, 2, 3, 4]):
    # a useful function from Stackoverflow, to count particular weekdays in date range
    if isinstance(start, dt.datetime):
        start = start.date()               # make a date from a datetime

    if isinstance(end, dt.datetime):
        end = end.date()                   # make a date from a datetime

    if end < start:
        # you can opt to return 0 or swap the dates around instead
        # raise ValueError("start date must be before end date")
        end, start = start, end

    if inclusive:
        end += dt.timedelta(days=1)  # correct for inclusivity

    try:
        # collapse duplicate weekdays
        weekdays = {weekday % 7 for weekday in weekdays}
    except TypeError:
        weekdays = [weekdays % 7]
        
    ref = dt.date.today()                    # choose a reference date
    ref -= dt.timedelta(days=ref.weekday())  # and normalize its weekday

    return sum((ref_plus - start).days // 7 - (ref_plus - end).days // 7
               for ref_plus in
               (ref + dt.timedelta(days=weekday) for weekday in weekdays))

def countDaysInIntervalHelper(calendarrow):
    # function to find number of days of service operation based on calendars.txt-entry
    servicepattern = calendarrow.loc["monday":"sunday"].to_numpy()
    servicedays = servicepattern.nonzero()[0].tolist()

    startdate = dt.datetime.strptime(str(int(calendarrow.get("start_date"))),"%Y%m%d")
    enddate = dt.datetime.strptime(str(int(calendarrow.get("end_date"))),"%Y%m%d")
    return(interveningWeekdays(startdate, enddate, weekdays = servicedays))

### Helper function to compare dates
def isInIntervalHelper(n, interval):
    '''works only on ARRAY-like n'''
    return(np.where((n <= max(interval)) & (n >= min(interval)), True, False))

In [59]:
# function to add frequencies...
def addCountToCalendar(calendar_df, calendar_dates_df):
    # enriches stop_times DataFrame with information about how often in the feed
    # period each stop is made
    

    print("Getting number of service days for each service")
    # use service_id to find service...
    calendar_df["days_count"] = calendar_df.apply(countDaysInIntervalHelper, axis=1)    

    print("\t...aggregating calendar")
    calendar_df = calendar_dates_df.groupby(["service_id", "exception_type"], as_index=False
                              ).count(
                            ).pivot(index = "service_id", columns = "exception_type", values = "date"
                            ).reset_index(
                            ).merge(calendar_df, on="service_id", how="right"
                            )[['service_id', 1, 2, 'monday',
                                  'tuesday',  'wednesday',   'thursday',     'friday',   'saturday',
                                  'sunday', 'start_date',   'end_date', 'days_count']]
    
    print("\t...calculating total in calendar")
    calendar_df.days_count= (calendar_df.days_count + calendar_df[1].fillna(0) - calendar_df[2].fillna(0)
                            )
    
    return(calendar_df)

In [60]:
def feedDays(calendar_df, calendar_dates_df):
    ''' Enriches counted dataframe with average daily count for each stop,
    using the feed's calendar information to infer the number of days
    '''
    # calculate
    startdate =  min(pd.to_datetime(calendar_df.start_date,format="%Y%m%d"))
    enddate = max(pd.to_datetime(calendar_df.end_date,format="%Y%m%d"))
    excdates = pd.to_datetime(calendar_dates_df.date,format="%Y%m%d")

    firstdate = min(startdate, min(excdates))
    lastdate = max(enddate, max(excdates))

    ndays = (lastdate - firstdate).days
    
    return(ndays)

In [61]:
calendar_df = pd.read_csv(zf.open("calendar.txt"))
calendar_dates_df = pd.read_csv(zf.open("calendar_dates.txt"))

In [62]:
calendar_df = addCountToCalendar(calendar_df, calendar_dates_df)

Getting number of service days for each service
	...aggregating calendar
	...calculating total in calendar


In [63]:
ndays = feedDays(calendar_df, calendar_dates_df) # total number of days in feed period

# Pick out routes

The below only makes sense if you are filtering a feed by routes. We did this with the gtfs.de dataset, using a partial FV-Export to determine which routes to include.

In that case, make sure the correct zip file with the routes you want to include is listed below.

!! Set filter_by_route-flag up top to determine whether this happens

In [64]:
if filter_by_route:
    fv_routes = pd.read_csv(zipfile.ZipFile(gtfsdir + "fv_211028.zip"
                        ).open("routes.txt"))

    routes_df = pd.read_csv(zf.open("routes.txt"))

    routes_df = fv_routes[["route_long_name"]
                    ].merge(routes_df, how="inner",on="route_long_name"
                    ).sort_values("route_id")

# Get things into database

## calendar

In [65]:
# put enriched calendar into database
calendar_df.to_sql("calendar", 'sqlite:///' + outpath,
          if_exists = 'replace')

## routes

In [66]:
# if filtering by routes, put routes file into database
if filter_by_route == True:
    routes_df.to_sql("routes", 'sqlite:///' + outpath,
              if_exists = 'replace')

## trips, stops, et al.

Transfer gtfs-files into database in chunks

In [ ]:
%%time
start = dt.datetime.now()
chunksize = 200000

if filter_by_route == True: # if we already wrote filtered routes
    ziptables = ['stops','trips', 'stop_times']
else: # otherwise, do it together with all the other tables
    ziptables = ['stops','trips', 'stop_times', 'routes']
    
    
for table_name in ziptables:
    print(table_name)

    j=0
    for df in pd.read_csv(zf.open(table_name + ".txt"),
                          chunksize=chunksize, iterator=True, encoding='utf-8',
                           dtype={'Unnamed: 0': 'float64',
                           'drop_off_type': 'object',
                           'pickup_type': 'object',
                           'stop_sequence': 'object',
                           'trip_id': 'object',
                           'stop_headsign': 'object'}
                         ):
        j+=1
        if j%10==0: # track progress visibly
            print('\t{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

        if j==1:
            df.to_sql(table_name, dbout, if_exists='replace')
        else:
            df.to_sql(table_name, dbout, if_exists='append')

stops
trips
	145 seconds: completed 2000000 rows
	182 seconds: completed 4000000 rows
	218 seconds: completed 6000000 rows
	257 seconds: completed 8000000 rows
	294 seconds: completed 10000000 rows
	328 seconds: completed 12000000 rows
	360 seconds: completed 14000000 rows
	397 seconds: completed 16000000 rows
	435 seconds: completed 18000000 rows
stop_times
	512 seconds: completed 2000000 rows
	556 seconds: completed 4000000 rows
	599 seconds: completed 6000000 rows
	644 seconds: completed 8000000 rows
	686 seconds: completed 10000000 rows
	726 seconds: completed 12000000 rows
	768 seconds: completed 14000000 rows
	809 seconds: completed 16000000 rows
	849 seconds: completed 18000000 rows
	892 seconds: completed 20000000 rows
	932 seconds: completed 22000000 rows
	974 seconds: completed 24000000 rows
	1016 seconds: completed 26000000 rows
	1057 seconds: completed 28000000 rows
	1100 seconds: completed 30000000 rows
	1141 seconds: completed 32000000 rows
	1182 seconds: completed 340000

# Database Merging

Count stop_times per stop using SQL (to keep large files out of working memory)

    CREATE TABLE n_stops AS
       ...> SELECT stop_id, SUM(days_count)
       ...> FROM stop_times
       ...> LEFT JOIN trips ON trips.trip_id = stop_times.trip_id
       ...> LEFT JOIN calendar ON trips.service_id = calendar.service_id
       ...> GROUP BY stop_id;


This is where the filtering by route happens--therefore two different queries, one for filtering by route and one for including all stops.

In [ ]:
%%time
print(dt.datetime.now())

if filter_by_route:
    out_df = pd.read_sql_query(
        'SELECT n_stops.stop_id, n, stop_name, parent_station, stop_lat, stop_lon, location_type '
        'FROM ('
            'SELECT stop_id, SUM(days_count) AS n '
            'FROM ('
                'SELECT routes.route_short_name, routes.route_id, trips.service_id, trips.trip_headsign, trips.direction_id, trips.trip_id '
                'FROM routes '
                'LEFT JOIN trips ON routes.route_id = trips.route_id '
            ') AS trips_fv '
            'LEFT JOIN stop_times ON trips_fv.trip_id = stop_times.trip_id '
            'LEFT JOIN calendar ON trips_fv.service_id = calendar.service_id '
            'GROUP BY stop_id '
        ') AS n_stops '
        'JOIN stops ON n_stops.stop_id = stops.stop_id',
        dbout
    )

else:
    out_df = pd.read_sql_query(
        'SELECT n_stops.stop_id, n, stop_name, parent_station, stop_lat, stop_lon, location_type '
        'FROM ('
            'SELECT stop_id, SUM(days_count) AS n '
            'FROM trips '
            'LEFT JOIN stop_times ON trips.trip_id = stop_times.trip_id '
            'LEFT JOIN calendar ON trips.service_id = calendar.service_id '
            'GROUP BY stop_id '
        ') AS n_stops '
        'JOIN stops ON n_stops.stop_id = stops.stop_id',
        dbout
    )

# Wrap up and write out

In [ ]:
out_df["n_day"] = out_df.n/ndays # the count per day, for comparing different length feeds

In [ ]:
if filter_by_route:
    out_df.to_csv(outdir + zipname + ".fv.nstops.csv")
else:
    out_df.to_csv(outdir + zipname + ".nstops.csv")